<a href="https://colab.research.google.com/github/Lakshika-Swarnamali/Diagrams/blob/master/FYP_MNF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use ('fivethirtyeight')
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import collections

import tweepy as tw
import nltk
from nltk.corpus import stopwords
import re
import networkx

import warnings
warnings.filterwarnings("ignore")

sns.set(font_scale=1.5)
sns.set_style("whitegrid")
# packages to store and manipulate data
import pandas as pd
import numpy as np

# plotting packages
import matplotlib.pyplot as plt
import seaborn as sns

# model building package
import sklearn

# package to clean text
import re

In [ ]:
log = pd.read_csv("https://raw.githubusercontent.com/GihanKLG/TwitterCovid19Dataset/master/2020-10-12.csv",encoding='latin1')
log[(log['lang'] == 'en')].to_csv("out.csv", index=False)
log = pd.read_csv('out.csv', lineterminator='\n',encoding='latin1')
log.head()

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,retweet_count,retweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_screen_name,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name.1,user_statuses_count,user_time_zone,user_urls,user_verified
0,NaN,Sun Oct 11 23:59:54 +0000 2020,NaN,NaN,NaN,0,1315442032651927552,BalutSmithy,1.315403e+18,1.296441e+18,en,NaN,NaN,0,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",@BalutSmithy @StephenJohn59 @jaketapper @kaitl...,https://twitter.com/NorthLapp/status/131544203...,Sun Mar 25 19:25:10 +0000 2012,NorthLapp,False,"I need more followers, I will expose the truth...",1979,1291,1805,28,NaN,Renee J North Lapp,NorthLapp,12503,NaN,NaN,False
1,NaN,Sun Oct 11 23:59:56 +0000 2020,NaN,NaN,NaN,0,1315442037286678529,NaN,NaN,NaN,en,NaN,NaN,1,1.315348e+18,PetraCEsser1,"<a href=""https://mobile.twitter.com"" rel=""nofo...",RT @PetraCEsser1: @HSRetoucher Thank you. I am...,https://twitter.com/johncoppa44/status/1315442...,Fri Feb 26 09:25:23 +0000 2016,johncoppa44,True,NaN,3031,31,287,1,NaN,john coppa,johncoppa44,7490,NaN,NaN,False
2,NaN,Sun Oct 11 23:59:56 +0000 2020,NaN,NaN,https://twitter.com/onlinelonghorn/status/1315...,0,1315442039870324736,NaN,NaN,NaN,en,NaN,False,0,NaN,NaN,"<a href=""http://twitter.com/download/iphone"" r...",*corona liked this post* https://t.co/3mAPNWNAfK,https://twitter.com/ivythekingrat/status/13154...,Mon Dec 11 16:09:37 +0000 2017,ivythekingrat,False,nope,4001,95,153,0,puerto rico,rat zone,ivythekingrat,1633,NaN,NaN,False
3,NaN,Sun Oct 11 23:59:55 +0000 2020,SidNaaz,NaN,NaN,0,1315442036028260353,NaN,NaN,NaN,en,NaN,NaN,120,1.315334e+18,SidNaaz13Dstny,"<a href=""http://twitter.com/download/android"" ...",RT @SidNaaz13Dstny: Points\n\n1. Salman Sir #S...,https://twitter.com/Tejakshi1/status/131544203...,Sat Jan 04 06:50:54 +0000 2020,Tejakshi1,False,Ã°ÂÂ¤Â¡,57815,282,579,0,NaN,tejakshi shetty,Tejakshi1,39117,NaN,NaN,False
4,NaN,Sun Oct 11 23:59:55 +0000 2020,NaN,NaN,NaN,0,1315442033331507203,NaN,NaN,NaN,en,NaN,NaN,10711,1.315345e+18,CLewandowski_,"<a href=""http://twitter.com/#!/download/ipad"" ...",RT @CLewandowski_: WHO now advising against lo...,https://twitter.com/LoisR52511816/status/13154...,Sun Aug 02 16:10:47 +0000 2020,LoisR52511816,True,NaN,28886,88,277,0,NaN,LoisR,LoisR52511816,13602,NaN,NaN,False


In [ ]:
# importing pandas package
import pandas as pd
 
# making data frame from csv file
data = pd.read_csv("https://raw.githubusercontent.com/GihanKLG/TwitterCovid19Dataset/master/2020-10-12.csv", index_col ="id")
 
# retrieving columns by indexing operator
# first = data["user_name"]
my_column_names = [ 'user_name','text']
df = pd.DataFrame(data=data, columns=my_column_names)
lda = pd.DataFrame(data=data, columns=my_column_names)
documents = pd.DataFrame(data=data, columns=my_column_names)

# print(first)
# print(first['id', 'user_name','retweet_count','retweet_id'])
print(df)
# df.head(5)

                              user_name                                               text
id                                                                                        
1315442032651927552  Renee J North Lapp  @BalutSmithy @StephenJohn59 @jaketapper @kaitl...
1315442033507684352             🎃 leh 🎃  RT @dougraz: todo dia pensando será se eu ja p...
1315442037286678529          john coppa  RT @PetraCEsser1: @HSRetoucher Thank you. I am...
1315442040172425216      Reivindicación               @PedroAzzola https://t.co/RxL0DAaQaI
1315442039870324736            rat zone   *corona liked this post* https://t.co/3mAPNWNAfK
...                                 ...                                                ...
1315457129835302914     AnnieMoonIsBlue  RT @BillKristol: On President Trump and Dr. Fa...
1315457130460327938        Dardo Boggia  RT @osvaldo_quiroga: Lo entrevisté muchísimas ...
1315457129847889921                Shay    ISOLATION\n\nNot COVID. https://t.co/d4j0jm7j0e

In [ ]:
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def remove_links(text):
    '''Takes a string and removes web links from it'''
    text = re.sub(r'http\S+', '', text) # remove http links
    text = re.sub(r'bit.ly/\S+', '', text) # rempve bitly links
    text = text.strip('[link]') # remove [links]
    return text

def remove_users(text):
    '''Takes a string and removes retweet and @user information'''
    text = re.sub('(RT\s@[A-Za-z]+[A-Za-z0-9-_]+)', '', text) # remove retweet
    text = re.sub('(@[A-Za-z]+[A-Za-z0-9-_]+)', '', text) # remove tweeted at
    return text

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'

# cleaning master function
def clean_tweet(text, bigrams=False):
    text = remove_users(text)
    text = remove_links(text)
    text = text.lower() # lower case
    text = re.sub('['+my_punctuation + ']+', ' ', text) # strip punctuation
    text = re.sub('\s+', ' ', text) #remove double spacing
    text = re.sub('([0-9]+)', '', text) # remove numbers
    tweet_token_list = [word for word in text.split(' ')
                            if word not in my_stopwords] # remove stopwords

    tweet_token_list = [word_rooter(word) if '#' not in word else word
                        for word in tweet_token_list] # apply word rooter
    if bigrams:
        tweet_token_list = tweet_token_list+[tweet_token_list[i]+'_'+tweet_token_list[i+1]
                                            for i in range(len(tweet_token_list)-1)]
    tweet = ' '.join(tweet_token_list)
    return tweet

In [ ]:
documents['clean_tweet'] = documents.text.apply(clean_tweet)
documents.head()

,user_name,text,clean_tweet
id,,,
1315442032651927552,Renee J North Lapp,@BalutSmithy @StephenJohn59 @jaketapper @kaitl...,seem fauci also blame mishandl pandem birx pr...
1315442033507684352,🎃 leh 🎃,RT @dougraz: todo dia pensando será se eu ja p...,todo dia pensando será se eu ja peguei covid ...
1315442037286678529,john coppa,RT @PetraCEsser1: @HSRetoucher Thank you. I am...,thank speak tongu angel bring chip blood brai...
1315442040172425216,Reivindicación,@PedroAzzola https://t.co/RxL0DAaQaI,
1315442039870324736,rat zone,*corona liked this post* https://t.co/3mAPNWNAfK,corona like post


In [ ]:
# use tfidf by removing tokens that don't appear in at least 50 documents
vect = TfidfVectorizer(min_df=50, stop_words='english')
 
# Fit and transform
X = vect.fit_transform(documents.clean_tweet)

In [ ]:
# Create an NMF instance: model
# the 10 components will be the topics
model = NMF(n_components=10, random_state=5)
 
# Fit the model to TF-IDF
model.fit(X)
 
# Transform the TF-IDF: nmf_features
nmf_features = model.transform(X)

In [ ]:
X.shape

(166536, 3802)

In [ ]:
nmf_features.shape

(166536, 10)

In [ ]:
model.components_.shape

(10, 3802)

In [ ]:
# Create a DataFrame: components_df
components_df = pd.DataFrame(model.components_, columns=vect.get_feature_names())
components_df

,ab,aban,abc,abil,abl,abo,abort,absolut,abuja,abus,aca,acab,acaba,acabar,accept,access,accident,accion,accord,account,accur,accus,acerca,achiev,acknowledg,act,action,activ,activo,actor,actual,actualización,acuerdo,acumulado,acá,ad,ada,adalah,adam,adapt,...,วน,วนจากโคว,วย,สถานศ,สทำร,สอดประกาศให,หนวกถาวรไปข,หล,หายท,อนข,อนเด,อว,อเคสผ,าง,าทำไมข,ายห,าวน,าวล,าไวร,เจอเคสห,เช,เด,เร,เศษ,เอะอะก,แต,แม,แล,ใช,ในประเทศม,ในว,ไปเจอผ,ๆๆๆ,메신저,사랑의,평화와,𝐂𝐎𝐍𝐓𝐈𝐆𝐎,𝐄𝐒,𝙄𝙣𝙛𝙤,𝙧𝙢𝙖𝙩𝙚
0,0.000000,0.000000,0.000000,0.000425,1.768226e-03,0.000000,7.810381e-04,2.676618e-03,0.000000,9.496168e-04,0.000652,0.000000,0.000000,0.000000,1.993130e-03,0.001339,0.000000,0.000000,2.222069e-02,0.001955,5.755285e-04,0.000502,0.000000,1.883796e-04,0.000000,2.145653e-03,2.153082e-03,0.000000e+00,0.000000,0.000068,3.849573e-03,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000059,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000059,0.000009,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000059,0.000000,0.000000,0.000000,0.000266,0.000266,0.000000,0.000000
1,0.004426,0.003426,0.011679,0.000954,1.339907e-04,0.000000,1.635612e-03,0.000000e+00,0.000000,4.357375e-04,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000291,0.000000,0.000000,0.000000e+00,0.000299,0.000000e+00,0.004937,0.000000,9.151420e-04,0.000000,6.689695e-04,0.000000e+00,5.851620e-04,0.000000,0.000000,2.970812e-03,0.000000,0.000000e+00,0.000000,0.000000,2.570722,0.000000,0.000262,0.000000,0.000305,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.001049,2.331705e-03,0.000000,9.815260e-06,1.067821e-03,0.000149,2.589653e-04,0.001259,0.000000,0.000000,0.000000,2.931688e-04,0.001086,0.000000,0.000000,0.000000e+00,0.001287,5.484563e-04,0.000000,0.000000,1.412434e-04,0.000000,3.092409e-03,3.470145e-03,3.045954e-02,0.000000,0.000604,2.955888e-03,0.000000,0.000000e+00,0.000000,0.000000,0.009910,0.000815,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.004735,0.018839,0.001959,0.012621,5.279686e-02,0.002242,8.273523e-03,3.453425e-02,0.003677,9.597005e-03,0.080053,0.010138,0.000000,0.000654,1.133753e-02,0.011238,0.020881,0.003699,2.674725e-02,0.020284,1.892808e-02,0.013518,0.001331,5.017754e-03,0.065318,3.950119e-02,4.151173e-02,1.489942e-01,0.008215,0.005140,5.437807e-02,0.014600,0.000000e+00,0.004683,0.000000,0.000000,0.032043,0.005485,0.009096,0.007870,...,0.035749,0.000058,0.006667,0.035749,0.000058,0.035749,0.000058,0.016784,0.000058,0.000058,0.000539,0.000058,0.006667,0.000058,0.006667,0.000058,0.006667,0.000058,0.000494,0.000058,0.001466,0.002753,0.036,0.035749,0.000539,0.002614,0.037204,0.037722,0.003847,0.006667,0.035773,0.006667,0.000539,0.000000,0.000000,0.000000,0.000000,0.000000,0.002519,0.002519
4,0.000000,0.000000,0.004608,0.000000,0.000000e+00,0.000056,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.038081,0.026876,0.012306,0.000000e+00,0.000000,0.005667,0.044605,0.000000e+00,0.000000,0.000000e+00,0.000000,0.036904,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.037790,0.016012,2.689921e-03,0.104244,3.172564e-02,0.025949,0.045128,0.000412,0.000457,0.000181,0.000000,0.000000,...,0.000000,0.000000,0.000

In [ ]:
for topic in range(components_df.shape[0]):
    tmp = components_df.iloc[topic]
    print(f'For topic {topic+1} the words with the highest value are:')
    print(tmp.nlargest(10))
    print('\n')

For topic 1 the words with the highest value are:
doe         2.694841
wear        2.686319
woman       2.685614
straight    2.683249
choic       2.679073
save        2.663883
mask        2.661486
person      2.617069
let         2.563171
live        2.542178
Name: 0, dtype: float64


For topic 2 the words with the highest value are:
context     2.581577
ad          2.570722
pathet      2.469364
wit         2.455725
comment     2.453204
latest      2.446828
use         2.365979
amp         2.293940
campaign    2.246847
fauci       2.245425
Name: 1, dtype: float64


For topic 3 the words with the highest value are:
song        4.679170
hear        4.287498
pandem      3.344844
graham      0.108669
lindsey     0.107556
chair       0.103243
tomorrow    0.102755
advanc      0.097645
trum        0.097048
refus       0.090840
Name: 2, dtype: float64


For topic 4 the words with the highest value are:
covid       7.313813
test        1.371351
neg         0.845082
peopl       0.826336
case    